In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from scripts import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: d62c0dde-aae2-48e9-9f17-b20839a711a2
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [5]:
from llama_index import Document
# join it into 1 single list
document = Document(text="\n\n".join([doc.text for doc in documents]))

## Window-sentence retrieval setup

In [6]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [7]:
text = "hello. how are you? I am fine!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [8]:
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [9]:
print(nodes[1].metadata["window"])

hello.  how are you?  I am fine!  


In [10]:
# node_parser = SentenceWindowNodeParser.from_defaults(
#     window_size=2,
#     window_metadata_key="window",
#     original_text_metadata_key="original_text",
# )

# text = "hello. foo bar. cat dog. mouse"

# nodes = node_parser.get_nodes_from_documents([Document(text=text)])
# print([x.text for x in nodes])
# print(nodes[0].metadata["window"])

### Building the index

In [11]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [12]:
from llama_index import ServiceContext
# vector store created( it need the llm, embedding model and node parser)
sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    # embed_model="local:BAAI/bge-large-en-v1.5"
    node_parser=node_parser,
)

In [13]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [14]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")


In [15]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context
    )

### Building the post processor

In [16]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [17]:
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [18]:
from llama_index.schema import NodeWithScore
from copy import deepcopy

# replicating similarity search result
scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

In [19]:
nodes_old[1].text

'how are you? '

In [20]:
replaced_nodes = postproc.postprocess_nodes(scored_nodes)

In [21]:
print(replaced_nodes[1].text)

hello.  how are you?  I am fine!  


### Adding a reranker

In [22]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=3, model="BAAI/bge-reranker-base"
)

In [23]:
## a reranker uses question and document as input  for similarity analysis

### The cell below is used to test a query with its scores and returns a correct answer

In [24]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
    NodeWithScore(node=TextNode(text="A dog is domestic animal"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a lion"), score=0.4),
]

In [25]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

In [26]:
print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.91827345), ('A dog is domestic animal', 0.6579715), ('This is a lion', 0.001732392)]


### Running the query engine

In [27]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

In [28]:
window_response = sentence_window_engine.query(
    "What are the keys to building a career in AI?"
)

In [29]:
from llama_index.response.notebook_utils import display_response

display_response(window_response)

**`Final Response:`** The keys to building a career in AI are discussed in Chapter 10 of the book.

### Putting it all together

In [30]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [31]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)


In [32]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)


In [33]:
from pprint import pprint 
pprint(query_engine.query("what are the main skills needed to build a carrer in AI").response)

('The main skills needed to build a career in AI include learning foundational '
 'technical skills, working on projects, and finding a job. Additionally, '
 'being part of a supportive community can also be beneficial. It is important '
 'to stay curious and continue learning, even if something may not seem '
 'immediately useful. The ability to identify ideas worth working on is also a '
 'valuable skill for an AI architect.')


### TruLens Evaluation

In [34]:
eval_questions = []
with open('text/generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [35]:
from trulens_eval import Tru

tru = Tru(database_url="sqlite:///db/sentence_window.sqlite")


🦑 Tru initialized with db url sqlite:///sentence_window.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [36]:

from llama_index.core.base_query_engine import BaseQueryEngine
from trulens_eval import TruLlama
from typing import List

def run_evals(eval_questions: List[str], tru_recorder, query_engine: BaseQueryEngine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [37]:
from scripts.utils import get_prebuilt_trulens_recorder



### Sentence window size = 1

In [38]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1",
)

In [39]:
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)

In [40]:
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1'
)

In [41]:
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

In [43]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

RuntimeError: Dashboard failed to start in time. Please inspect dashboard logs for additional information.

### Sentence window size = 3

In [ ]:
sentence_index_3 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3",
)

In [ ]:
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

In [ ]:
tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)

In [ ]:
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)

Error calling wrapped function get_response.
Traceback (most recent call last):
  File "c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\trulens_eval\instruments.py", line 732, in tru_wrapper
    rets, cost = Endpoint.track_all_costs_tally(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\trulens_eval\feedback\provider\endpoint\base.py", line 462, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\trulens_eval\feedback\provider\endpoint\base.py", line 411, in track_all_costs
    return Endpoint._track_costs(thunk, with_endpoints=endpoints)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\trulens_eval\feedback\provider\endpoint\base.py", line 568, in _track_costs
    result: T = thunk()
                ^^^^^^^
  File 

KeyboardInterrupt: 

In [ ]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.74:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
# tru.get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
sentence window engine 3,1.0,0.925000,0.85,22.0,0.001383
sentence window engine 1,1.0,0.422222,0.70,22.0,0.001053


- put all steps in window size put it in a single  function where sentence widow size is the only parameter taht changes and use string formatting to change the app id and test for sentence window sixze of 5

In [ ]:
def build_run_evals(window_size):
    sentence_index = build_sentence_window_index(
        documents,
        llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
        embed_model="local:BAAI/bge-small-en-v1.5",
        sentence_window_size=window_size,
        save_dir=f"sentence_index_{window_size}"
    )
    
    sentence_window_engine = get_sentence_window_query_engine(sentence_index)
    
    tru_recorder = get_prebuilt_trulens_recorder(
        sentence_window_engine,
        app_id=f'sentence window engine {window_size}'
    )
    
    run_evals(eval_questions, tru_recorder, sentence_window_engine)

In [ ]:
build_run_evals(5)

In [ ]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
sentence window engine 5,1.0,1.000000,0.90,22.0,0.001715
sentence window engine 3,1.0,0.925000,0.85,22.0,0.001383
sentence window engine 1,1.0,0.422222,0.70,22.0,0.001053


In [ ]:
build_run_evals(7)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1dff4e13090 is calling an instrumented method <function BaseQueryEngine.query at 0x000001DFAFCEAC00>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1dffc2ea4d0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1dff4e13090 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001DFB5FE0220>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1dffc2ea4d0) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x1dff4e13f10 is calling an instrumented method <function BaseRetriever.retrieve at 0x000001DFB2743100>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on ot

In [ ]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
sentence window engine 5,1.0,1.000000,0.90,22.0,0.001715
sentence window engine 7,1.0,0.928571,0.90,22.0,0.002123
sentence window engine 3,1.0,0.925000,0.85,22.0,0.001383
sentence window engine 1,1.0,0.422222,0.70,22.0,0.001053


In [ ]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.74:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [44]:
tru.stop_dashboard()

ValueError: Dashboard not running in this workspace. You may be able to shut other instances by setting the `force` flag.